In [2]:
# 데이터 준비
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

0.9973541965122431 0.8905151032797809


In [3]:
# - 랜덤 포레스트 -

# 100개의 트리 사용, 각 노드에서 선택할 특성의 개수 = 특성의 개수 제곱근 사용 (기본값)
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# return_train_score = 훈련 성능 점수 반환
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [4]:
rf.fit(train_input, train_target)
# 특성의 중요도 출력
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [5]:
# 데이터 세트 중 트리를 만들 때 사용하는 샘플이 아닌 것 = oob
# oob 데이터를 활용하여 각 트리의 예측 성능 평가 = oob_score
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [6]:
# - 엑스트라 트리 -

# 100개의 트리 사용, 각 노드에서 선택할 특성의 개수 = 특성의 개수 제곱근 사용 (기본값)
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [7]:
et.fit(train_input, train_target)
print(et.feature_importances_)

# 트리의 성장 억제 방법
# 랜덤 포레스트 = 부트스트랩 샘플링(추출된 데이터를 다시 데이터셋에 추가) 사용 / 각 트리를 구성할 때 원본 데이터에서 부트스트랩 샘플(복원 추출된 데이터)을 생성하여 사용 
# = 데이터의 일부를 랜덤하게 선택함으로써 다양성을 증가, 과적합을 줄이는 효과
# 엑스트라 트리 = 부트스트랩 샘플링 X, 완전한 데이터 사용 / 노드를 더욱 랜덤하게 분할 (무작위성 부여)

[0.20183568 0.52242907 0.27573525]


In [8]:
# - 그레디언트 부스팅 -
# 손실함수 = 분류 : 로지스틱 손실함수(예측 확률과 실제 클래스 사이의 차이를 최소화) / 회귀 : 평균제곱오차 (예측 값과 실제 값 사이의 제곱 오차를 최소화) 사용
# 손실함수가 최소화 되도록 회귀 트리(결정트리 회귀모델)를 계속해서 추가

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [9]:
# n_estimator = 트리의 개수 조절, 이전 트리의 오차를 줄이는 방향으로 생성 / 과다할 시 과적합 문제
# learning_rate = 이전 트리의 오차 보정 강도 조절 / 각 트리의 기여도를 조절하여 모델의 학습 속도와 정확도 사이의 균형을 조절하는 데 사용 / 기본값 = 0.1
# = 낮을 수록 적은 양의 보정 -> 많은 시간 소요 but 과적합 위험도 저하 / 높을 수록 많은 양의 보정 -> 적은 시간 소요 but 과적합 위험도 증가
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [10]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


In [14]:
# - 히스토그램 기반 그레디언트 부스팅 -
# 256개의 구간으로 분할 255개는 훈련에 사용, 1개는 누락된 값에 할당 -> 따라서 누락된 값에 대한 전처리 불필요
# 데이터를 256개의 구간으로 이산화하여 속도 향상, 메모리 효율성 증가 / 특성의 값의 범위를 제한하기 때문에 특성의 값이 많은 경우 효과적

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [15]:
# - permutation importance -
# 치환 중요도 : 특성 중 하나를 무작위로 섞어 성능 평가, 원래 성능과 비교하여 특성 중요도 파악

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
# n_repeats = 특성 치환 수행 횟수 조절
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [16]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [17]:
hgb.score(test_input, test_target)

0.8723076923076923

In [19]:
# - XGBoost -
# 히스토그램 기반 그레디언트 부스팅 지원

from xgboost import XGBClassifier

# tree_method = 'hist' 히스토그램 기반 그레이언트 부스팅 모델 사용
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [20]:
# - LightGBM -
# 히스토그램 기반 그레디언트 부스팅 지원

from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
